In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import seaborn as sns


import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler,MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.linear_model import LogisticRegression,LinearRegression 
from xgboost import XGBClassifier,XGBRegressor
%matplotlib inline
from sklearn.neighbors import KNeighborsRegressor
import datetime
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.ensemble import VotingRegressor

In [ ]:
train = pd.read_csv(r'/kaggle/input/bike-share/TRAIN.csv')
test = pd.read_csv(r'/kaggle/input/bike-share/TEST.csv')

test1 = test.copy()

df = pd.concat([train,test])

df['Day'] = pd.to_datetime(df.Date, format='%d/%m/%Y').dt.day            
df['Month'] = pd.to_datetime(df.Date, format='%d/%m/%Y').dt.month 
df['Year'] = pd.to_datetime(df.Date, format='%d/%m/%Y').dt.year
df['dow'] = pd.to_datetime(df.Date, format='%d/%m/%Y').dt.dayofweek
df['woy'] = pd.to_datetime(df.Date, format='%d/%m/%Y').dt.weekofyear

df = df.drop('Index', axis = 1)

cols= ['Seasons', 'Holiday','Functioning Day']
df[cols]= df[cols].apply(LabelEncoder().fit_transform)

In [ ]:
columns =['Hour', 'Temperature(°C)', 'Humidity(%)', 'Wind speed (m/s)',
       'Visibility (10m)', 'Dew point temperature(°C)',
       'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)', 'Seasons',
       'Holiday', 'Functioning Day', 'Rented Bike Count', 'Day', 'Month']
for i in columns:
    print(i, df[i].skew())

In [ ]:
df['Weekday'] =  1
df['Weekday'] = ((pd.DatetimeIndex(df['Date']).dayofweek) // 5 == 0).astype(float)

In [ ]:
sns.histplot(x = df['Wind speed (m/s)'], y= df['Rented Bike Count'])

In [ ]:
sns.histplot(x = df['Rainfall(mm)'], y= df['Rented Bike Count'])

In [ ]:
sns.distplot(x = df['Rainfall(mm)'])

In [ ]:
plt.figure(figsize=(12,10))
sns.histplot(x = df['Temperature(°C)'], y= df['Rented Bike Count'],data=df [df['Weekday']==1])

In [ ]:
plt.figure(figsize=(12,10))
sns.histplot(x = df['Temperature(°C)'], y= df['Rented Bike Count'],data=df [df['Weekday']==0])

In [ ]:
plt.figure(figsize =(12,14))
sns.boxplot(x= 'Temperature(°C)',y='Rented Bike Count', data=df)

In [ ]:
df['goodW']=0
df.loc[((df["Temperature(°C)"] >24)&(df['Temperature(°C)'] <28)&(df['Wind speed (m/s)'] <3)&(df['Rainfall(mm)'] ==0)),"goodW"] =1

In [ ]:
df[df['goodW']==1]

In [ ]:
plt.figure(figsize =(12,14))
sns.boxplot(x= 'Hour',y='Rented Bike Count', data=df, hue = 'Weekday')
plt.title('year')

In [ ]:
sns.boxplot(x= 'Seasons',y='Rented Bike Count', data=df)
plt.title('Seasons')

In [ ]:
plt.figure(figsize = (12,10))
sns.boxplot(x= 'Month',y='Rented Bike Count', data=df)
plt.title('Months')

In [ ]:
plt.figure(figsize = (12,10))
sns.boxplot(x= 'Month',y='Rented Bike Count', data=df[df['Weekday']==0])
plt.title('Months weekend')

In [ ]:
plt.figure(figsize = (12,10))
sns.boxplot(x= 'Month',y='Rented Bike Count', data=df[df['Weekday']==1])
plt.title('Months weekend')

In [ ]:

sns.boxplot(x= 'Holiday',y='Rented Bike Count', data=df)
plt.title('Months')

In [ ]:
plt.figure(figsize = (12,10))
sns.boxplot(x= 'Hour',y='Rented Bike Count', data=df[df['Holiday']==0])
plt.title('Houre Holiday')

In [ ]:
plt.figure(figsize = (12,10))
sns.boxplot(x= 'Hour',y='Rented Bike Count', data=df)
plt.title('Houre')

In [ ]:
plt.figure(figsize = (12,10))
sns.boxplot(x= 'Hour',y='Rented Bike Count', data=df[df['Weekday']==1])
plt.title('Houre weekday')

In [ ]:
plt.figure(figsize = (12,10))
sns.boxplot(x= 'Hour',y='Rented Bike Count', data=df[df['Weekday']==0])
plt.title('Houre weekend')